In [ ]:
"""
from google.colab import drive
drive.mount('/content/drive')"""

Mounted at /content/drive


In [44]:
#!pip install pymatreader

In [1]:
import pandas as pd
import numpy as np

In [2]:
def kinz_data(data):
  numbodies_list = data["kinzData"]['numbodies']
  req_index = []

  for i in range(len(numbodies_list)):
    if numbodies_list[i]==1:
      req_index.append(i)


  kinz_arr = np.empty([0,97])

  for i in req_index:

    timeStamps = np.array(data["kinzData"]['timeStamps'][i])
    kinz_x = data["kinzData"]['bodies'][i]['Position3d'][0]
    kinz_y = data["kinzData"]['bodies'][i]['Position3d'][1]
    kinz_z = data["kinzData"]['bodies'][i]['Position3d'][2]

    d = np.concatenate((timeStamps,data["kinzData"]['bodies'][i]['Position3d'][0], data["kinzData"]['bodies'][i]['Position3d'][1],
                        data["kinzData"]['bodies'][i]['Position3d'][2]), axis=None)


    kinz_arr = np.append(kinz_arr, [d], axis=0)



  arr_walking_speed = np.array([data["walking_speed"]]*len(req_index))
  arr_trial_num = np.array([data["trial_num"]]*len(req_index))
  arr_patientID = np.array([data["patientID"]]*len(req_index))

  #res_kinz_arr = np.column_stack((arr_patientID,arr_trial_num,arr_walking_speed,kinz_arr))
  return np.column_stack((arr_patientID,arr_trial_num,arr_walking_speed,kinz_arr))






def acc_data(data):

  arr_walking_speed = np.array([data["walking_speed"]]*len(data["accelData"]['timeStamps']))
  arr_stopwatch = np.array([data["stopwatch"]]*len(data["accelData"]['timeStamps']))
  arr_trial_num = np.array([data["trial_num"]]*len(data["accelData"]['timeStamps']))
  arr_patientID = np.array([data["patientID"]]*len(data["accelData"]['timeStamps']))

  #res = np.column_stack((arr_patientID,arr_trial_num,arr_walking_speed,arr_stopwatch,data["accelData"]['timeStamps'],data["accelData"]['timeSeries']))
  return np.column_stack((arr_patientID,arr_trial_num,arr_walking_speed,arr_stopwatch,data["accelData"]['timeStamps'],data["accelData"]['timeSeries']))





import numpy as np

def kinz_time(kinz_data):
    frames_num = len(kinz_data)
    time_in_sec = np.zeros(frames_num)
    time0 = kinz_data[0]  # time zero for the acquisition

    for i in range(frames_num):
        time_dbl = kinz_data[i] - time0  # relative time (double precision)
        time_in_sec[i] = time_dbl / 1e9  # Convert time to seconds.

    mean_time_diff = np.mean(np.diff(time_in_sec))  # Mean time difference
    fs = 1 / mean_time_diff  # Sampling rate

    return time_in_sec, fs

In [3]:
from pymatreader import read_mat
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm

data_path = '/home/iqh4001/Iqram_WCM/MADE Lab/Kinect Project/Dataset/Data/Cleaned Data'

#data_path = "/content/drive/MyDrive/Iqram Sir/AI_Frailty-MATLAB_Sample"

In [ ]:
# for all mat file

"""all_file_path=[]

for i in sorted(glob(data_path+"/*")):
  all_file_path = all_file_path+glob(i+"/*.mat")
  print(i)"""

In [5]:
import os
#for clean preproccessed data
all_file_path=[]

for i in sorted(glob(data_path+"/*")):
    for j in glob(i+"/*"):
        if 'Cln' in os.path.basename(j).split("-"):
            all_file_path.append(j)

In [15]:
path = "/home/iqh4001/Iqram_WCM/MADE Lab/Kinect Project/Dataset/Data/Cleaned Data/015-IL/Cln-015-IL_Trial4_Fast_20220406_093218.mat"

read_mat(path)

/home/iqh4001/.conda/envs/jupyter-notebook/lib/python3.10/site-packages/pymatreader/utils.py:238: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '


{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Wed Sep  7 15:35:33 2022',
 '__version__': '1.0',
 '__globals__': [],
 'accelData': {'timeSeries': array([[ 2.27530580e-04,  2.59763031e-04, -1.00367961e-04, ...,
           1.70331255e-04,  7.59324633e-07,  9.67801598e-06],
         [ 2.32798837e-04,  2.54594237e-04, -1.08450255e-04, ...,
           1.62274174e-04,  9.51943488e-07,  9.74738443e-06],
         [ 2.38069254e-04,  2.49135288e-04, -1.16193873e-04, ...,
           1.54932949e-04,  1.14423179e-06,  9.78745438e-06],
         ...,
         [ 1.05345136e-04,  1.81035471e-04,  4.36332607e-05, ...,
          -2.83584814e-03,  5.49198564e-06, -3.38298374e-05],
         [ 1.03413464e-04,  1.80106166e-04,  4.32304594e-05, ...,
          -2.81285856e-03,  5.58639388e-06, -3.36684492e-05],
         [ 1.01904217e-04,  1.79321963e-04,  4.28933627e-05, ...,
          -2.79412190e-03,  5.66176730e-06, -3.35381496e-05]]),
  'timeStamps': array([-2.97615469, -2.9760961 , -2

In [14]:
import os
from datetime import datetime

current_datetime = datetime.now()
formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M%p")

file_save_destination = '/home/iqh4001/Iqram_WCM/MADE Lab/Kinect Project/Output Data/Kinect Data/kinzData_panda_'+formatted_datetime+'.csv'

#file_save_destination = "task1.csv"


if os.path.exists(file_save_destination)==False:
  pd.DataFrame(np.empty([0,100]), columns=['patientID','trial_num','walking_speed','timeStamps',
  'PELVIS_X', 'SPINE_NAVAL_X', 'SPINE_CHEST_X', 'NECK_X', 'CLAVICLE_LEFT_X', 'SHOULDER_LEFT_X', 'ELBOW_LEFT_X', 'WRIST_LEFT_X', 'HAND_LEFT_X', 'HANDTIP_LEFT_X', 'THUMB_LEFT_X', 'CLAVICLE_RIGHT_X', 'SHOULDER_RIGHT_X', 'ELBOW_RIGHT_X', 'WRIST_RIGHT_X', 'HAND_RIGHT_X', 'HANDTIP_RIGHT_X', 'THUMB_RIGHT_X', 'HIP_LEFT_X', 'KNEE_LEFT_X', 'ANKLE_LEFT_X', 'FOOT_LEFT_X', 'HIP_RIGHT_X', 'KNEE_RIGHT_X', 'ANKLE_RIGHT_X', 'FOOT_RIGHT_X', 'HEAD_X', 'NOSE_X', 'EYE_LEFT_X', 'EAR_LEFT_X', 'EYE_RIGHT_X', 'EAR_RIGHT_X',
  'PELVIS_Y', 'SPINE_NAVAL_Y', 'SPINE_CHEST_Y', 'NECK_Y', 'CLAVICLE_LEFT_Y', 'SHOULDER_LEFT_Y', 'ELBOW_LEFT_Y', 'WRIST_LEFT_Y', 'HAND_LEFT_Y', 'HANDTIP_LEFT_Y', 'THUMB_LEFT_Y', 'CLAVICLE_RIGHT_Y', 'SHOULDER_RIGHT_Y', 'ELBOW_RIGHT_Y', 'WRIST_RIGHT_Y', 'HAND_RIGHT_Y', 'HANDTIP_RIGHT_Y', 'THUMB_RIGHT_Y', 'HIP_LEFT_Y', 'KNEE_LEFT_Y', 'ANKLE_LEFT_Y', 'FOOT_LEFT_Y', 'HIP_RIGHT_Y', 'KNEE_RIGHT_Y', 'ANKLE_RIGHT_Y', 'FOOT_RIGHT_Y', 'HEAD_Y', 'NOSE_Y', 'EYE_LEFT_Y', 'EAR_LEFT_Y', 'EYE_RIGHT_Y', 'EAR_RIGHT_Y',
  'PELVIS_Z', 'SPINE_NAVAL_Z', 'SPINE_CHEST_Z', 'NECK_Z', 'CLAVICLE_LEFT_Z', 'SHOULDER_LEFT_Z', 'ELBOW_LEFT_Z', 'WRIST_LEFT_Z', 'HAND_LEFT_Z', 'HANDTIP_LEFT_Z', 'THUMB_LEFT_Z', 'CLAVICLE_RIGHT_Z', 'SHOULDER_RIGHT_Z', 'ELBOW_RIGHT_Z', 'WRIST_RIGHT_Z', 'HAND_RIGHT_Z', 'HANDTIP_RIGHT_Z', 'THUMB_RIGHT_Z', 'HIP_LEFT_Z', 'KNEE_LEFT_Z', 'ANKLE_LEFT_Z', 'FOOT_LEFT_Z', 'HIP_RIGHT_Z', 'KNEE_RIGHT_Z', 'ANKLE_RIGHT_Z', 'FOOT_RIGHT_Z', 'HEAD_Z', 'NOSE_Z', 'EYE_LEFT_Z', 'EAR_LEFT_Z', 'EYE_RIGHT_Z', 'EAR_RIGHT_Z'
                                                ]).to_csv(file_save_destination,index=False)



#try:

for path in tqdm(all_file_path):
  data = read_mat(path)
  temp_df = pd.DataFrame(kinz_data(data), columns=['patientID','trial_num','walking_speed','timeStamps',
  'PELVIS_X', 'SPINE_NAVAL_X', 'SPINE_CHEST_X', 'NECK_X', 'CLAVICLE_LEFT_X', 'SHOULDER_LEFT_X', 'ELBOW_LEFT_X', 'WRIST_LEFT_X', 'HAND_LEFT_X', 'HANDTIP_LEFT_X', 'THUMB_LEFT_X', 'CLAVICLE_RIGHT_X', 'SHOULDER_RIGHT_X', 'ELBOW_RIGHT_X', 'WRIST_RIGHT_X', 'HAND_RIGHT_X', 'HANDTIP_RIGHT_X', 'THUMB_RIGHT_X', 'HIP_LEFT_X', 'KNEE_LEFT_X', 'ANKLE_LEFT_X', 'FOOT_LEFT_X', 'HIP_RIGHT_X', 'KNEE_RIGHT_X', 'ANKLE_RIGHT_X', 'FOOT_RIGHT_X', 'HEAD_X', 'NOSE_X', 'EYE_LEFT_X', 'EAR_LEFT_X', 'EYE_RIGHT_X', 'EAR_RIGHT_X',
  'PELVIS_Y', 'SPINE_NAVAL_Y', 'SPINE_CHEST_Y', 'NECK_Y', 'CLAVICLE_LEFT_Y', 'SHOULDER_LEFT_Y', 'ELBOW_LEFT_Y', 'WRIST_LEFT_Y', 'HAND_LEFT_Y', 'HANDTIP_LEFT_Y', 'THUMB_LEFT_Y', 'CLAVICLE_RIGHT_Y', 'SHOULDER_RIGHT_Y', 'ELBOW_RIGHT_Y', 'WRIST_RIGHT_Y', 'HAND_RIGHT_Y', 'HANDTIP_RIGHT_Y', 'THUMB_RIGHT_Y', 'HIP_LEFT_Y', 'KNEE_LEFT_Y', 'ANKLE_LEFT_Y', 'FOOT_LEFT_Y', 'HIP_RIGHT_Y', 'KNEE_RIGHT_Y', 'ANKLE_RIGHT_Y', 'FOOT_RIGHT_Y', 'HEAD_Y', 'NOSE_Y', 'EYE_LEFT_Y', 'EAR_LEFT_Y', 'EYE_RIGHT_Y', 'EAR_RIGHT_Y',
  'PELVIS_Z', 'SPINE_NAVAL_Z', 'SPINE_CHEST_Z', 'NECK_Z', 'CLAVICLE_LEFT_Z', 'SHOULDER_LEFT_Z', 'ELBOW_LEFT_Z', 'WRIST_LEFT_Z', 'HAND_LEFT_Z', 'HANDTIP_LEFT_Z', 'THUMB_LEFT_Z', 'CLAVICLE_RIGHT_Z', 'SHOULDER_RIGHT_Z', 'ELBOW_RIGHT_Z', 'WRIST_RIGHT_Z', 'HAND_RIGHT_Z', 'HANDTIP_RIGHT_Z', 'THUMB_RIGHT_Z', 'HIP_LEFT_Z', 'KNEE_LEFT_Z', 'ANKLE_LEFT_Z', 'FOOT_LEFT_Z', 'HIP_RIGHT_Z', 'KNEE_RIGHT_Z', 'ANKLE_RIGHT_Z', 'FOOT_RIGHT_Z', 'HEAD_Z', 'NOSE_Z', 'EYE_LEFT_Z', 'EAR_LEFT_Z', 'EYE_RIGHT_Z', 'EAR_RIGHT_Z'
                                                ])

  columns_to_convert = temp_df.columns.difference(['patientID','trial_num', 'walking_speed'])
  temp_df[columns_to_convert] = temp_df[columns_to_convert].astype(float)

  k = kinz_time(temp_df["timeStamps"])
  temp_df["timeStamps"] = k[0]
  print(os.path.basename(path))
  print(kinz_data(data)[0][0],temp_df['patientID'][0])
  #temp_df.to_csv(file_save_destination, mode='a', index=False, header=False)
  print("_________________")

"""except:
  print("The Subject folder or main path unusual file or folder below.....Please remove the file or the folder.....")
  print(path)"""

  0%|          | 0/309 [00:00<?, ?it/s]

/home/iqh4001/.conda/envs/jupyter-notebook/lib/python3.10/site-packages/pymatreader/utils.py:238: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '
/home/iqh4001/.conda/envs/jupyter-notebook/lib/python3.10/site-packages/pymatreader/utils.py:238: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '


Cln-001-LO_Trial1_Regular_20220307_111113.mat
001-LO 001-LO
_________________
Cln-001-LO_Trial2_Regular_20220307_111139.mat
001-LO 001-LO
_________________


/home/iqh4001/.conda/envs/jupyter-notebook/lib/python3.10/site-packages/pymatreader/utils.py:238: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '
/home/iqh4001/.conda/envs/jupyter-notebook/lib/python3.10/site-packages/pymatreader/utils.py:238: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '


Cln-001-LO_Trial3_Fast_20220307_111208.mat
001-LO 001-LO
_________________
Cln-001-LO_Trial4_Fast_20220307_111234.mat
001-LO 001-LO
_________________


...

Cln-080-TW_Trial1_Regular_20231117_060933.mat
080-TW 080-TW
_________________
Cln-080-TW_Trial2_Regular_20231117_060958.mat
080-TW 080-TW
_________________


/home/iqh4001/.conda/envs/jupyter-notebook/lib/python3.10/site-packages/pymatreader/utils.py:238: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '
/home/iqh4001/.conda/envs/jupyter-notebook/lib/python3.10/site-packages/pymatreader/utils.py:238: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '


Cln-080-TW_Trial3_Fast_20231117_061019.mat
080-TW 080-TW
_________________
Cln-080-TW_Trial4_Fast_20231117_061135.mat
080-TW 080-TW
_________________


/home/iqh4001/.conda/envs/jupyter-notebook/lib/python3.10/site-packages/pymatreader/utils.py:238: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '


'except:\n  print("The Subject folder or main path unusual file or folder below.....Please remove the file or the folder.....")\n  print(path)'

In [13]:
os.path.basename(path)

'Cln-001-LO_Trial1_Regular_20220307_111113.mat'

In [11]:
kinz_data(data)[0][0]

'001-LO'

In [28]:
import os
from datetime import datetime

current_datetime = datetime.now()
formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M%p")

file_save_destination_1 = '/home/iqh4001/Iqram_WCM/MADE Lab/Kinect Project/Output Data/ACC Data/ACC_Data_panda_'+formatted_datetime+'.csv'

#file_save_destination_1 = "task2.csv"

if os.path.exists(file_save_destination_1)==False:
  pd.DataFrame(np.empty([0,12]), columns=['patientID','trial_num','walking_speed','stopwatch','timeStamps','acc_1', 'acc_2', 'acc_3','acc_4', 'acc_5', 'acc_6','acc_7']).to_csv(file_save_destination_1,index=False)




for path in tqdm(all_file_path):
  data = read_mat(path)
  temp_df = pd.DataFrame(acc_data(data), columns=['patientID','trial_num','walking_speed','stopwatch','timeStamps','acc_1', 'acc_2', 'acc_3',
                                  'acc_4', 'acc_5', 'acc_6','acc_7'])


  columns_to_convert = temp_df.columns.difference(['patientID','trial_num','walking_speed','stopwatch'])
  temp_df[columns_to_convert] = temp_df[columns_to_convert].astype(float)

  k = kinz_time(temp_df["timeStamps"])
  temp_df["timeStamps"] = k[0]

  temp_df.to_csv(file_save_destination_1, mode='a', index=False, header=False)



  0%|          | 0/309 [00:00<?, ?it/s]

/home/iqh4001/.conda/envs/jupyter-notebook/lib/python3.10/site-packages/pymatreader/utils.py:238: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '
/home/iqh4001/.conda/envs/jupyter-notebook/lib/python3.10/site-packages/pymatreader/utils.py:238: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '
/home/iqh4001/.conda/envs/jupyter-notebook/lib/python3.10/site-packages/pymatreader/utils.py:238: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '
/home/iqh4001/.conda/envs/jupyter-notebook/lib/python3.10/site-packages/pymatreader/utils.py:238: UserWarning: Complex objects (like 